In [1]:
from pynq import Overlay
base_overlay = Overlay("system_wrapper.xsa")

In [2]:
base_overlay?

In [4]:
help(base_overlay.axi_uartlite_0)

Help on DefaultIP in module pynq.overlay object:

class DefaultIP(builtins.object)
 |  DefaultIP(description)
 |  
 |  Driver for an IP without a more specific driver
 |  
 |  This driver wraps an MMIO device and provides a base class
 |  for more specific drivers written later. It also provides
 |  access to GPIO outputs and interrupts inputs via attributes. More specific
 |  drivers should inherit from `DefaultIP` and include a
 |  `bindto` entry containing all of the IP that the driver
 |  should bind to. Subclasses meeting these requirements will
 |  automatically be registered.
 |  
 |  Attributes
 |  ----------
 |  mmio : pynq.MMIO
 |      Underlying MMIO driver for the device
 |  _interrupts : dict
 |      Subset of the PL.interrupt_pins related to this IP
 |  _gpio : dict
 |      Subset of the PL.gpio_dict related to this IP
 |  
 |  Methods defined here:
 |  
 |  __init__(self, description)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  read(se

In [27]:
from pynq import DefaultIP
import time

class MyUartDriver(DefaultIP):
    RX_FIFO_ADDR = 0x0
    TX_FIFO_ADDR = 0x4
    STAT_REG_ADDR = 0x8
    CTRL_REG_ADDR = 0xC
    
    def __init__(self, description):
        super().__init__(description=description)
    
    bindto = ['xilinx.com:ip:axi_uartlite:2.0']
    
    def write_data(self,data):
        """ Writes a string to the UART one byte at a time."""
        for char in data:
            one_byte = ord(char)
            while self.get_stat()["Tx_FIFO_Full"]:
                time.sleep(0.01)
            self.write(self.TX_FIFO_ADDR, one_byte)
            
            
    def get_stat(self):
        """ Return the UART status register info as a dictionary """
        
        stat = self.read(self.STAT_REG_ADDR)
        status = {
            "Parity_Error"       : bool(stat & 0b10000000),
            "Frame_Error"        : bool(stat & 0b01000000),
            "Overrun_Error"      : bool(stat & 0b00100000),
            "Interrupt_Enabled"  : bool(stat & 0b00010000),
            "Tx_FIFO_Full"       : bool(stat & 0b00001000),
            "Tx_FIFO_Empty"      : bool(stat & 0b00000100),
            "Rx_FIFO_Full"       : bool(stat & 0b00000010),
            "Rx_FIFO_Valid_Data" : bool(stat & 0b00000001)
        }
        return status
    
    def read_data(self):
        """ Reads data from the UART's RX FIFO until it's empty."""
        data =[]
        while self.get_stat()["Rx_FIFO_Valid_Data"]:
            byte = self.read(self.RX_FIFO_ADDR)
            data.append(chr(byte))
        return ''.join(data)
    
    def set_interrupt(self, enable):
        """ set the UART interrupt based on the 'enable' flag. """
        current_value = self.read(self.CTRL_RG_ADDR)
        if enable:
            new_value = current_value | (1<<4)
        else:
            new_value = current_value & ~(1<<4)
        self.write(self.CTRL_REG_ADDR, new_value)
            
    def rest_tx_fifo(self):
        self._reset_fifo('tx')
    
    def reset_rx_fifo(self):
        self._reset_fifo('rx')
        
    def _reset_fifo(self, fifo_type):
        """reset the specified FIFO ('tx' for transmit FIFO or 'rx' for receive FIFO)."""
        bit_position = 0 if fifo_type =='tx' else 1
        
        current_value = self.read(self.CTRL_REG_ADDR)
        new_value = current_value | (1<<bit_position)
        self.write(self.CTRL_ADDR, new_value)
        
        
        current_value = self.read(self.CTRL_REG_ADDR)
        new_value = current_value & ~(1<<bit_position)
        self.write(self.CTRL_ADDR, new_value)
        

In [28]:
base_overlay= Overlay("./system_wrapper.xsa")
uart = base_overlay.axi_uartlite_0
help(uart)

Help on MyUartDriver in module __main__ object:

class MyUartDriver(pynq.overlay.DefaultIP)
 |  MyUartDriver(description)
 |  
 |  Method resolution order:
 |      MyUartDriver
 |      pynq.overlay.DefaultIP
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, description)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  get_stat(self)
 |      Return the UART status register info as a dictionary
 |  
 |  read_data(self)
 |      Reads data from the UART's RX FIFO until it's empty.
 |  
 |  reset_rx_fifo(self)
 |  
 |  rest_tx_fifo(self)
 |  
 |  set_interrupt(self, enable)
 |      set the UART interrupt based on the 'enable' flag.
 |  
 |  write_data(self, data)
 |      Writes a string to the UART one byte at a time.
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  CTRL_REG_ADDR = 12
 |  
 |  RX_FIFO_ADDR = 0
 |  
 |  STAT_REG_ADDR = 8
 |  
 |  TX_

In [34]:
uart.get_stat()

{'Parity_Error': False,
 'Frame_Error': True,
 'Overrun_Error': False,
 'Interrupt_Enabled': False,
 'Tx_FIFO_Full': False,
 'Tx_FIFO_Empty': True,
 'Rx_FIFO_Full': False,
 'Rx_FIFO_Valid_Data': False}

In [35]:
uart.write_data("Hello World!")

In [36]:
time.sleep(5)

In [37]:
data = uart.read_data()

In [38]:
print(data)

Hello World!
